<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/paligemma_cnmc_finetune_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paligemma Fine Tuning using CNMC dataset

### Setting Up

In [1]:
base_model_name = "google/paligemma-3b-pt-224"
adapter_version = "paligemma-cnmc-ft"
adapter_model_name = f"dwb2023/{adapter_version}"

In [2]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
import io
import os
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import(
    PaliGemmaForConditionalGeneration, AutoProcessor, TrainingArguments,
    Trainer, BitsAndBytesConfig, BitsAndBytesConfig
)
from datasets import load_dataset, DatasetDict

In [4]:
# verify gpu is available
torch.cuda.is_available()

True

In [5]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [6]:
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

## Load Dataset

In [7]:
# Load CNMC Dataset
ds = load_dataset("dwb2023/cnmc-leukemia-2019", split="train")

Generating train split:   0%|          | 0/10661 [00:00<?, ? examples/s]

In [8]:
# Load CNMC Dataset
ds = load_dataset("dwb2023/cnmc-leukemia-2019", split="train")

# Filter records to only include those from fold 0
ds_fold_0 = ds.filter(lambda example: example['fold'] == 0)

# Define the percentage you want to retrieve (e.g., 10%)
percentage = 0.10

# Use train_test_split to get the subset
cnmc_ds = ds_fold_0.train_test_split(test_size=percentage)["test"]

# Columns to remove
cols_remove = ["subject_id", "image_number", "cell_count", "class_label", "fold", "original_image_name", "relative_file_path"]
cnmc_ds = cnmc_ds.remove_columns(cols_remove)

Filter:   0%|          | 0/10661 [00:00<?, ? examples/s]

In [9]:
# create train test split with test_size=0.2
train_ds = cnmc_ds.train_test_split(test_size=0.2)

# create test val split
test_val_ds = train_ds["test"].train_test_split(test_size=0.5)

cnmc_ds_dict = DatasetDict({
    "train" : train_ds["train"],
    "test" : test_val_ds["test"],
    "validation" : test_val_ds["train"]
})

cnmc_ds_dict

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 282
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 36
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 35
    })
})

## Collate Data

In [23]:
# Define Dataset Preprocessor
class DatasetPreprocessor:
    def __init__(self, processor, max_seq_length):
        self.processor = processor
        self.max_seq_length = max_seq_length

    def preprocess_function(self, examples):
        texts = ["Are these cells healthy or cancerous?" for _ in range(len(examples['image']))]
        images = []
        for img in examples['image']:
            try:
                image = Image.open(io.BytesIO(img['bytes'])).convert("RGB")
                images.append(image)
            except UnidentifiedImageError as e:
                print(f"Unidentified image: {img['path']} - Error: {e}")
                images.append(None)
            except Exception as e:
                print(f"Error processing image: {img['path']} - Error: {e}")
                images.append(None)

        labels = examples['label']

        # Filter out None images if no placeholder image is provided
        valid_indices = [i for i, img in enumerate(images) if img is not None]
        valid_images = [img for img in images if img is not None]
        valid_texts = [texts[i] for i in valid_indices]
        valid_labels = [labels[i] for i in valid_indices]

        if not valid_images:
            return {}  # Return an empty dict if no valid images

        inputs = self.processor(
            text=valid_texts,
            images=valid_images,
            return_tensors="pt",
            padding="max_length",
            max_length=self.max_seq_length,
            truncation=True
        )
        label_inputs = self.processor(
            text=valid_labels,
            images=valid_images,
            return_tensors="pt",
            padding="max_length",
            max_length=self.max_seq_length,
            truncation=True
        )
        inputs['labels'] = label_inputs.input_ids

        return inputs

    def preprocess_dataset(self, dataset):
        # First map the dataset with the preprocessing function
        processed_dataset = dataset.map(
            self.preprocess_function,
            batched=True,
            remove_columns=dataset.column_names
        )

        # Then filter out any empty results
        processed_dataset = processed_dataset.filter(lambda x: len(x['input_ids']) > 0)

        return processed_dataset

In [11]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(base_model_name)

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [12]:
max_seq_length = 128
output_dir = adapter_version

In [18]:
from PIL import Image
import io

def collate_fn(examples):
    texts = ["Are these cells healthy or cancerous?" for _ in range(len(examples))]
    labels = [example['label'] for example in examples]
    images = [Image.open(io.BytesIO(example['image']['bytes'])).convert("RGB") for example in examples]
    tokens = processor(text=texts, images=images, return_tensors="pt", padding="longest")
    tokens["labels"] = processor.tokenizer(labels, padding="longest", return_tensors="pt").input_ids
    tokens = tokens.to(torch.bfloat16).to("cuda")
    return tokens

## Load and Quatize the base Model (bitsandbytes)

In [14]:
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_type=torch.bfloat16
)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model = PaliGemmaForConditionalGeneration.from_pretrained(base_model_name, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
#trainable params: 11,298,816 || all params: 2,934,634,224 || trainable%: 0.38501616002417344


Unused kwargs: ['bnb_4bit_compute_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


## Train the Adapter Model (trl)

In [15]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PaliGemmaForConditionalGeneration(
      (vision_tower): SiglipVisionModel(
        (vision_model): SiglipVisionTransformer(
          (embeddings): SiglipVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
            (position_embedding): Embedding(256, 1152)
          )
          (encoder): SiglipEncoder(
            (layers): ModuleList(
              (0-26): 27 x SiglipEncoderLayer(
                (self_attn): SiglipAttention(
                  (k_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=1152, out_features=1152, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1152, out_features=8, bias=False)
                    )
                    (lora_B): Mo

In [16]:
model.config

PaliGemmaConfig {
  "_name_or_path": "google/paligemma-3b-pt-224",
  "architectures": [
    "PaliGemmaForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "eos_token_id": 1,
  "hidden_size": 2048,
  "ignore_index": -100,
  "image_token_index": 257152,
  "model_type": "paligemma",
  "pad_token_id": 0,
  "projection_dim": 2048,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "text_config": {
    "hidden_size": 2048,
    "intermediate_size": 16384,
    "model_type": "gemma",
    "num_attention_heads": 8,
    "num_hidden_layers": 18,
    "num_image_tokens": 256,

In [17]:
from trl import SFTConfig

args=SFTConfig(
    output_dir = output_dir,
    max_steps=50,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=5,
    learning_rate=2e-5,
    max_seq_length=max_seq_length,
    bf16=True,
    optim="adamw_hf",
    report_to=["tensorboard"],
)

In [24]:
# Instantiate the preprocessor
preprocessor = DatasetPreprocessor(processor, max_seq_length=512)

# Preprocess the datasets
train_dataset = preprocessor.preprocess_dataset(cnmc_ds_dict['train'])
eval_dataset = preprocessor.preprocess_dataset(cnmc_ds_dict['validation'])

Map:   0%|          | 0/282 [00:00<?, ? examples/s]

Filter:   0%|          | 0/282 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Filter:   0%|          | 0/35 [00:00<?, ? examples/s]

In [25]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [26]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyError: 'label'

In [ ]:
trainer.push_to_hub(f"dwb2023/{output_dir}")

### Temporary code below to assist with debugging Image Key error

In [20]:
print(f"cnmc_ds_dict['train'][0]['image'].keys():\n{cnmc_ds_dict['train'][0]['image'].keys()}")
print(f"\ncnmc_ds_dict['train'][0]['image']['path']:\n{cnmc_ds_dict['train'][0]['image']['path']}")
print(f"\ncnmc_ds_dict['train'][0]['image']['bytes'][0:250]:\n{cnmc_ds_dict['train'][0]['image']['bytes'][0:250]}")

cnmc_ds_dict['train'][0]['image'].keys():
dict_keys(['bytes', 'path'])

cnmc_ds_dict['train'][0]['image']['path']:
UID_48_28_3_all.bmp

cnmc_ds_dict['train'][0]['image']['bytes'][0:250]:
b'BM\xc6H\t\x00\x00\x00\x00\x006\x00\x00\x00(\x00\x00\x00\xc2\x01\x00\x00\xc2\x01\x00\x00\x01\x00\x18\x00\x00\x00\x00\x00\x90H\t\x00\xc3\x0e\x00\x00\xc3\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
ds[0].keys()